# Using PLUR to implement zonemaps

The point of this notebook is to show how zonemaps can be implemented _using_ PLUR data structures, which would make it easier to use the resulting selection _in_ PLUR.

It also illustrates how PLUR data can be logically restructured without physically moving or changing the underlying data. In particular, partitioning a dataset is essentially free (an $O(n)$ operation where $n$ is the number of partitions and $N$ is the number of entries, with $n \ll N$).

In [1]:
from math import *

import numpy

from plur.types import *
from plur.python.data import *

## Getting the dataset

Get some arrays representing a few columns from a dataset. These can be made with the `avro2plur` script.

In [2]:
# pick a sample; naturally, this link to my home directory won't work for you
directory = "/home/pivarski/storage/data/TTJets_allevents/0/"

# "AK4CHS" are jets, "Muon" are muons
arrays = {n: numpy.load(directory + n + ".npy") for n in ("events-Lo", "events-Ld-R_Muon-Lo", "events-Ld-R_Muon-Ld-R_pt", "events-Ld-R_Muon-Ld-R_eta", "events-Ld-R_Muon-Ld-R_phi", "events-Ld-R_AK4CHS-Lo", "events-Ld-R_AK4CHS-Ld-R_pt", "events-Ld-R_AK4CHS-Ld-R_eta", "events-Ld-R_AK4CHS-Ld-R_phi")}

# look at the names of what we picked
sorted(arrays.keys(), reverse=True)

['events-Lo',
 'events-Ld-R_Muon-Lo',
 'events-Ld-R_Muon-Ld-R_pt',
 'events-Ld-R_Muon-Ld-R_phi',
 'events-Ld-R_Muon-Ld-R_eta',
 'events-Ld-R_AK4CHS-Lo',
 'events-Ld-R_AK4CHS-Ld-R_pt',
 'events-Ld-R_AK4CHS-Ld-R_phi',
 'events-Ld-R_AK4CHS-Ld-R_eta']

Turn the arrays into a data structure called "events" whose schema is inferred from the names of the arrays.

In this notebook, we're going to use the (slow) pure Python methods to work with the data. These are lazy-evaluated proxies that behave the same way as the compiled code, but at this early stage, they're more likely to work (or at least debug). The compiled interface should work the same but is orders of magnitude faster.

In [3]:
events = fromarrays("events", arrays)
events

[<events at 0x0>, <events at 0x1>, <events at 0x2>, <events at 0x3>, ...]

Printing out the `events.type` requires PLUR 0.0.4.

In [4]:
print(formattype(events.type))

List(
  Record(
    AK4CHS = List(
      Record(
        eta = float32,
        phi = float32,
        pt = float32
        )
      ),
    Muon = List(
      Record(
        eta = float32,
        phi = float32,
        pt = float32
        )
      )
    )
  )


## Example user function

Define a user analysis function. This is a fairly typical example, which selects events if they have one good muon and two good jets with minimum $p_T$ cuts to define what "good" means. In this example, we return an event list to use as a skim for more work later.

In [5]:
def original(events):
    for eventindex, event in enumerate(events):
        goodmuon = False
        for muon in event.Muon:
            if muon.pt > 150:           # want at least one muon.pt > 150
                goodmuon = True
                break
        numgoodjets = 0
        for jet in event.AK4CHS:
            if jet.pt > 300:            # want at least two jet.pt > 300
                numgoodjets += 1
                if numgoodjets >= 2:
                    break
        if goodmuon and numgoodjets >= 2:
            yield eventindex

selected = list(original(events))
selected

[411,
 999,
 2360,
 4196,
 4348,
 5027,
 6334,
 9614,
 9943,
 10279,
 11124,
 14536,
 15172,
 17484,
 19247,
 24346,
 26348,
 27265,
 28183,
 28374,
 28390,
 29428,
 30516,
 30582,
 32876,
 33891,
 38366,
 39713]

Check the first selected event; it should have at least one (probably only one) high-$p_T$ muon and at least two (probably more) high-$p_T$ jets.

In [6]:
print([muon.pt for muon in events[selected[0]].Muon])
print([jet.pt for jet in events[selected[0]].AK4CHS])

[280.3884]
[386.17084, 324.08521, 291.20868, 136.7355, 67.818985, 59.253769, 53.791145, 50.497181, 26.028744, 16.0399]


This happens to be a highly selective cut, probably fewer than 0.1% of events pass.

In [23]:
100.0*len(selected)/len(events)

0.0701209586536776

## Making the zonemap

The main way this differs from a zonemap over a rectangular table is that we can't be precise about the number of zonemaps per page of data. In this example, we give each zonemap 100 events. In my sample, this is an _average_ of 117.2 muons and 775.2 jets (see below). All floating point values in this sample are 4 bytes, so a zonemap is about a tenth of a muon page and more than half a jet page (assuming 4 kB disk or CPU cache pages). It's sloppy.

In [19]:
print(numpy.mean([len(x.Muon) for x in events]))
print(numpy.mean([len(x.AK4CHS) for x in events]))

1.17244747189
7.75207232476


Instead of a fixed number of _events_ per zonemap, we could have grown each zone until it just barely exceeds a minimum number of _particles_. But then, which particle: muons or jets? Maybe jets because they're more abundant, so it's more critical that they (almost) line up with the page boundary? This method would be sloppy, too, but maybe less so, and that might lead to better average performance.

I don't think there's any way to cut zones exactly at the particle page boundaries (which are more relevant than event page boundaries because particles have more data), because zones must end at event boundaries and events have arbitrary numbers of particles.

The zones have to be made in a nested for loop, rather than Numpy commands, because we need to calculate such things as a maximum second-highest jet $p_T$ per event. The need for nested loops would be even more acute if we vary the number of events per zone to try to more tightly control the number of jets per zone, as described above.

Don't worry about this being slow: this is exactly the sort of loop that can be accelerated by PLUR code transformation.

In [8]:
ZONE_SIZE = 100    # 100 events per zone
NUM_ZONES = int(ceil(1.0 * len(events) / ZONE_SIZE))

# they don't align with page boundaries, but it's impossible to get them to
eventstarts = numpy.empty(NUM_ZONES + 1, dtype=numpy.int64)
eventstarts[:-1] = range(0, len(events), 100)
eventstarts[-1] = len(events)

# three quantities to index per zone
muonpt1max = numpy.ones(NUM_ZONES, dtype=numpy.float32) * -numpy.inf
jetpt1max = numpy.ones(NUM_ZONES, dtype=numpy.float32) * -numpy.inf
jetpt2max = numpy.ones(NUM_ZONES, dtype=numpy.float32) * -numpy.inf

for eventindex, event in enumerate(events):
    muonpt1 = 0.0
    for muon in event.Muon:
        if muon.pt > muonpt1:
            muonpt1 = muon.pt     # assume muons are unsorted (I don't know for sure)

    jetpt1 = 0.0
    jetpt2 = 0.0
    for jet in event.AK4CHS:
        if jet.pt > jetpt1:
            jetpt2 = jetpt1
            jetpt1 = jet.pt
        elif jet.pt > jetpt2:
            jetpt2 = jet.pt

    # fill zone arrays
    zoneindex = eventindex // 100
    if muonpt1 > muonpt1max[zoneindex]:
        muonpt1max[zoneindex] = muonpt1
    if jetpt1 > jetpt1max[zoneindex]:
        jetpt1max[zoneindex] = jetpt1
    if jetpt2 > jetpt2max[zoneindex]:
        jetpt2max[zoneindex] = jetpt2

Take a look at each of the index arrays. I only made maxima, rather than the usual min/max, because for $p_T$, we're never interested in the minimum.

In fact, it's rare for a physics cut to be both highly selective and also two-sided. The only examples I can think of are mass cuts around particle masses, but most of those cases are derived quantities defined by the user.

In [9]:
muonpt1max

array([   140.13899231,    122.19343567,    270.32687378,    142.27436829,
          280.38839722,    255.15280151,    221.03739929,     81.21549988,
         1994.28369141,    179.93135071,    135.44818115,    233.60491943,
          140.35559082,    208.70959473,    186.66424561,    160.04293823,
           87.51934052,    183.87934875,    934.67358398,    119.79482269,
          167.74356079,    423.80807495,     85.31292725,    544.70446777,
          354.73321533,    132.25041199,     97.14313507,    216.27388   ,
          121.09153748,    126.62540436,    233.45239258,    163.96498108,
         2601.79199219,    173.40885925,    167.02444458,  27280.78320312,
          141.15934753,     96.37037659,    138.74493408,    367.39672852,
          108.6539917 ,    152.61985779,    166.22453308,    346.74914551,
          135.96138   ,    111.14984131,    135.15914917,    239.34103394,
          133.22946167,    202.29946899,    396.62902832,    157.79870605,
          222.09028625,  

In [10]:
jetpt1max

array([  481.82681274,   271.77041626,   645.95025635,   346.60095215,
         446.91091919,   509.58755493,   431.06497192,   655.68341064,
         453.18643188,   486.47906494,   418.07223511,   317.51953125,
         476.60409546,   474.16946411,   368.95153809,   385.71835327,
         681.36132812,   604.85595703,   505.49127197,   601.6864624 ,
         457.70898438,   490.3269043 ,   498.96884155,   584.4239502 ,
         463.55877686,   370.59164429,   501.88726807,   601.71801758,
         576.88342285,   467.9937439 ,   492.88589478,   376.84777832,
         543.08465576,   489.2046814 ,   358.72564697,   518.20159912,
         451.73425293,   641.50756836,   430.44854736,   513.65820312,
         451.59146118,   849.15234375,   480.08099365,   881.81048584,
         395.32562256,   579.02398682,   655.3182373 ,   404.32778931,
         418.23254395,   407.1703186 ,   952.67944336,   449.98477173,
         598.57244873,   757.52685547,   362.72607422,   659.84039307,
      

In [11]:
jetpt2max

array([  244.5422821 ,   225.98408508,   340.81079102,   252.35488892,
         324.08520508,   258.53240967,   218.1619873 ,   313.1786499 ,
         308.15164185,   426.58319092,   403.72488403,   222.26753235,
         219.39265442,   341.10327148,   241.17784119,   366.39996338,
         595.5178833 ,   383.48156738,   402.06460571,   500.72235107,
         287.61526489,   395.94073486,   277.25350952,   397.98071289,
         321.26821899,   300.72467041,   347.72091675,   479.28955078,
         372.44476318,   351.06481934,   309.15100098,   254.4697876 ,
         415.87667847,   224.41099548,   263.78866577,   338.50494385,
         387.20999146,   449.92422485,   304.36291504,   311.89291382,
         389.37579346,   353.81420898,   370.050354  ,   644.515625  ,
         243.5422821 ,   205.82778931,   373.16213989,   356.69073486,
         326.86465454,   216.27587891,   501.20410156,   325.03442383,
         246.26774597,   566.55609131,   198.03089905,   355.46960449,
      

## Restructuring the data into zones

The next step is where PLUR really helps. We want to create a new, indexed dataset out of our old, unindexed dataset. The new dataset must partition the original events— changing the data structure— but we can do it without modifying the original arrays.

First, we gather the new arrays into a new dictionary, then put most of the old ones in this dictionary with modified names.

In [12]:
# create a new set of arrays
indexedarrays = {
  "zones-Lo":  numpy.array([0, NUM_ZONES], dtype=numpy.int64),
  "zones-Ld-R_muonpt1max": muonpt1max,
  "zones-Ld-R_jetpt1max":  jetpt1max,
  "zones-Ld-R_jetpt2max":  jetpt2max,
  "zones-Ld-R_events-Lo":  eventstarts}

# importing the original arrays without modification, apart from their names
for name, array in arrays.items():
    if name.startswith("events-Ld-"):
        indexedarrays[name.replace("events-Ld-", "zones-Ld-R_events-Ld-")] = array

In [13]:
# new dataset
zones = fromarrays("zones", indexedarrays)
zones

[<zones at 0x0>, <zones at 0x1>, <zones at 0x2>, <zones at 0x3>, ...]

In [14]:
# this is now a dataset of zones
len(zones)

400

The list of events is nested within the list of zones.

In [20]:
print(formattype(zones.type))

List(
  Record(
    events = List(
      Record(
        AK4CHS = List(
          Record(
            eta = float32,
            phi = float32,
            pt = float32
            )
          ),
        Muon = List(
          Record(
            eta = float32,
            phi = float32,
            pt = float32
            )
          )
        )
      ),
    jetpt1max = float32,
    jetpt2max = float32,
    muonpt1max = float32
    )
  )


In [15]:
zones[0].muonpt1max, zones[0].jetpt1max, zones[0].jetpt2max, zones[0].events

(140.13899,
 481.82681,
 244.54228,
 [<events at 0x0>, <events at 0x1>, <events at 0x2>, <events at 0x3>, ...])

And exactly 100 events are in each zone (except the last).

In [22]:
print([len(zone.events) for zone in zones])

[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,

## Using the zonemap

Now we modify the user's analysis function by putting it in a loop over zones. We only descend into the sub-loop over events if there are any matching particles.

The cut on `zone.jetpt2max` implies the cut on `zone.jetpt1max`, but if a tighter cut on `zone.jetpt1max` is desired, it can be explicit.

In [17]:
def modified(zones):
    for zonei, zone in enumerate(zones):
        if zone.muonpt1max > 150 and zone.jetpt2max > 300:
            for x in original(zone.events):
                yield zonei*ZONE_SIZE + x

selected2 = list(modified(zones))
selected2

[411,
 999,
 2360,
 4196,
 4348,
 5027,
 6334,
 9614,
 9943,
 10279,
 11124,
 14536,
 15172,
 17484,
 19247,
 24346,
 26348,
 27265,
 28183,
 28374,
 28390,
 29428,
 30516,
 30582,
 32876,
 33891,
 38366,
 39713]

Same final result, of course.

In [18]:
selected == selected2

True